In [1]:
!conda init bash
!conda activate py37_pytorch

no change     /anaconda/condabin/conda
no change     /anaconda/bin/conda
no change     /anaconda/bin/conda-env
no change     /anaconda/bin/activate
no change     /anaconda/bin/deactivate
no change     /anaconda/etc/profile.d/conda.sh
no change     /anaconda/etc/fish/conf.d/conda.fish
no change     /anaconda/shell/condabin/Conda.psm1
no change     /anaconda/shell/condabin/conda-hook.ps1
no change     /anaconda/lib/python3.7/site-packages/xontrib/conda.xsh
no change     /anaconda/etc/profile.d/conda.csh
no change     /home/natke/.bashrc
No action taken.

CommandNotFoundError: Your shell has not been properly configured to use 'conda activate'.
To initialize your shell, run

    $ conda init <SHELL_NAME>

Currently supported shells are:
  - bash
  - fish
  - tcsh
  - xonsh
  - zsh
  - powershell

See 'conda init --help' for more information and options.

IMPORTANT: You may need to close and restart your shell after running 'conda init'.




In [1]:
# From https://github.com/microsoft/onnxruntime/blob/master/orttraining/pytorch_frontend_examples/mnist_training.py
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np
import os

print(torch.__version__)

1.6.0.dev20200505+cu101


In [10]:
import onnxruntime
onnxruntime.__version__

'1.2.0'

In [2]:
from onnxruntime.capi.ort_trainer import IODescription, ModelDescription, ORTTrainer, ORTModel
from mpi4py import MPI

In [3]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [4]:
def my_loss(x, target):
    return F.nll_loss(F.log_softmax(x, dim=1), target)

In [5]:
def train_with_model(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        data = data.reshape(data.shape[0], -1)
        loss, pred = model.run(data, target)
        if batch_idx % 10 == 0:
            optimizer.step()
            optimizer.zero_grad()
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [6]:
def mnist_model_description():
    input_desc = IODescription('input1', ['batch', 784], torch.float32)
    label_desc = IODescription('label', ['batch', ], torch.int64, num_classes=10)
    loss_desc = IODescription('loss', [], torch.float32)
    probability_desc = IODescription('probability', ['batch', 10], torch.float32)
    return ModelDescription([input_desc, label_desc], [loss_desc, probability_desc])

In [11]:
use_cuda = torch.cuda.is_available()
torch.manual_seed(1)

kwargs = {'num_workers': 0, 'pin_memory': True}
    
train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=True, download=True,
                           transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=64, shuffle=True, **kwargs)
    

comm = MPI.COMM_WORLD
    
device = torch.device("cuda", 0) if use_cuda else torch.device("cpu")

n_gpu = 1

input_size = 784
hidden_size = 500
num_classes = 10
    
model = NeuralNet(input_size, hidden_size, num_classes)

model_desc = mnist_model_description()
    
# use log_interval as gradient accumulate steps
trainer = ORTTrainer(model,              # model
                     my_loss,            # loss function
                     model_desc,         # model description
                     "SGDOptimizer",     # optimizer name
                     None,               # optimizer attributes
                     IODescription('Learning_Rate', [1,], torch.float32),
                     device,             # device
                     _opset_version=12)  # opset version

for epoch in range(1, 11):
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        data = data.reshape(data.shape[0], -1)

        learning_rate = torch.tensor([0.01])
        loss = trainer.train_step(data, target, learning_rate)

        # Since the output corresponds to [loss_desc, probability_desc], the first value is taken as loss.
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss[0]))


#trainer.save_as_onnx('mnist.onnx')

AttributeError: module 'onnxruntime' has no attribute 'TrainingParameters'